In [27]:
import pandas as pd
import anthropic
import requests
from dotenv import load_dotenv
import base64

import csv
from io import BytesIO
from PIL import Image

load_dotenv()
client = anthropic.Anthropic()

In [28]:
def load_few_shot_examples(csv_file):
    examples = []
    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            examples.append({
                'product_name': row['Product_Name'],
                'product_images': eval(row['Product_Images']),
                'product_description': row['Product_Description']
            })
    return examples


def download_and_encode_image(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        image = Image.open(BytesIO(response.content))
        
        # Determine the correct media type
        if image.format.lower() == 'jpeg':
            media_type = 'image/jpeg'
        elif image.format.lower() == 'png':
            media_type = 'image/png'
        elif image.format.lower() == 'gif':
            media_type = 'image/gif'
        elif image.format.lower() == 'webp':
            media_type = 'image/webp'
        else:
            print(f"Unsupported image format: {image.format}")
            return None, None

        buffered = BytesIO()
        image.save(buffered, format=image.format)
        encoded_image = base64.b64encode(buffered.getvalue()).decode('utf-8')
        return encoded_image, media_type
    except Exception as e:
        print(f"Error downloading or encoding image: {e}")
        return None, None
    


In [41]:
def load_few_shot_pd(few_shot_examples):
    example_messages = []

    for example in few_shot_examples:
        # Prepare user message with product name and images
        user_content = [
            {
                "type": "text", 
                "text": f"Product Name: {example['product_name']}\nProduct Images:"
                }
            ]

        # Download and encode up to 5 valid images
        encoded_images = []
        for url in example['product_images']:
            if len(encoded_images) >= 5:
                break
            encoded_image, media_type = download_and_encode_image(url)
            if encoded_image and media_type:
                encoded_images.append((encoded_image, media_type))

        # Add encoded images to user content
        user_content.extend([
            {
                "type": "image",
                "source": {
                    "type": "base64",
                    "media_type": media_type,
                    "data": image_data
                }
            } for image_data, media_type in encoded_images
        ])

        # Add user message to example_messages
        example_messages.append({
            "role": "user",
            "content": user_content,
            "cache_control": {"type": "ephemeral"}
        })

        # Add assistant message (product description) to example_messages
        example_messages.append({
            "role": "assistant",
            "content": example['product_description']
        })

    return example_messages

In [42]:
few_shot_examples = load_few_shot_examples('examples/examples_sarcastic.csv')
example_messages = load_few_shot_pd(few_shot_examples[:3]) # load only 3 products to save money
example_messages

[{'role': 'user',
  'content': [{'type': 'text',
    'text': 'Product Name: Phoenix Knot Detail Suede Loafers\nProduct Images:'},
   {'type': 'image',
    'source': {'type': 'base64',
     'media_type': 'image/jpeg',
     'data': '...'}},
   {'type': 'image',
    'source': {'type': 'base64',
     'media_type': 'image/jpeg',
     'data': '...'}},
   {'type': 'image',
    'source': {'type': 'base64',
     'media_type': 'image/jpeg',
     'data': '...'}},
   {'type': 'image',
    'source': {'type': 'base64',
     'media_type': 'image/jpeg',
     'data': '...'}},
   {'type': 'image',
    'source': {'type': 'base64',
     'media_type': 'image/jpeg',
     'data': '...'}}],
  'cache_control': {'type': 'ephemeral'}},
 {'role': 'assistant',
  'content': 'Oh, joy! Behold these revolutionary loafers, featuring mind-blowing "knot detailing" and "gold-tone hardware" that will surely blind onlookers with their sheer magnificence. The groundbreaking "moc stitching" (because regular stitching is so pa

In [36]:
def process_row(row):
    # Prepare user message with product name and images
    user_content = [{"type": "text", "text": f"Product Name: {row['product_name']}\nProduct Images:"}]

    # Download and encode images
    encoded_images = []
    for url in row['product_images']:
        if len(encoded_images) >= 5:
            break
        encoded_image, media_type = download_and_encode_image(url)
        if encoded_image and media_type:
            encoded_images.append((encoded_image, media_type))

    # Add encoded images to user content
    user_content.extend([
        {
            "type": "image",
            "source": {
                "type": "base64",
                "media_type": media_type,
                "data": image_data
            }
        } for image_data, media_type in encoded_images
    ])

    # Prepare the messages for the API call
    messages = example_messages + [
        {"role": "user", "content": user_content}
    ]

    # Make the API call
    response = client.messages.create(
        model="claude-3-5-sonnet-20240620", # claude-3-5-sonnet-20240620
        max_tokens=1000,
        messages=messages,
        extra_headers={"anthropic-beta": "prompt-caching-2024-07-31"}
    )

    # Extract the generated description
    generated_description = response.content[0].text
    
    input_tokens = response.usage.input_tokens
    output_tokens = response.usage.output_tokens

    return generated_description, input_tokens, output_tokens, response

In [37]:
products = []
with open('demo/batch_1.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        products.append({
            'URL': row['URL'],
            'product_name': row['Product_Name'],
            'product_images': eval(row['Product_Images']),
        })

In [38]:
test = products[0]
process_row(test)

BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'messages.0.cache_control: Extra inputs are not permitted'}}

In [18]:
import time

for product in products[:3]:
    start_time = time.time()
    print(process_row(product))
    end_time = time.time()
    print(f"Time taken: {end_time - start_time} seconds")

('Ah, the Karina Studded Bow Ballet Flats - for when you want to look like you\'re trying way too hard to be both edgy and cute at the same time! These revolutionary flats combine the delicate grace of a ballet shoe with the subtlety of a rhinestone-encrusted sledgehammer. The dainty bow screams "I\'m still five years old at heart," while the studs along the edges whisper "but I might key your car if you cross me." \n\nPerfect for those days when you can\'t decide if you want to pirouette or start a bar fight. The pale pink color ensures you\'ll blend right in with all the other basic fashionistas, while the pointy toe offers a great way to stub your toe on every piece of furniture you own. And let\'s not forget the flat sole, because who needs arch support anyway? \n\nPair these bad boys with some white cropped pants and an off-shoulder top for the ultimate "I read one fashion blog and now I think I\'m stylish" look. Remember, nothing says sophistication quite like shoes that look lik

1. prompt caching
2. few shot with input(s)/output(s) 

In [8]:
import anthropic

client = anthropic.Anthropic()

response = client.beta.prompt_caching.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    system=[
      {
        "type": "text", 
        "text": "You are an AI assistant tasked with analyzing literary works. Your goal is to provide insightful commentary on themes, characters, and writing style.\n",
      },
      {
        "type": "text", 
        "text": "<the entire contents of 'Pride and Prejudice'>",
        "cache_control": {"type": "ephemeral"}
      }
    ],
    messages=[{"role": "user", "content": "Analyze the major themes in 'Pride and Prejudice'."}],
)
print(response)

PromptCachingBetaMessage(id='msg_01PpD1bLES7x2S6tj7GTJoFW', content=[TextBlock(text='Certainly! I\'d be happy to analyze the major themes in Jane Austen\'s "Pride and Prejudice" based on the text provided. Here are some of the key themes explored in the novel:\n\n1. Pride and Prejudice: As the title suggests, these two traits are central to the story. Many characters, especially Elizabeth Bennet and Mr. Darcy, must overcome their pride and preconceived notions about others to find happiness.\n\n2. Love and Marriage: The novel explores various motivations for marriage, contrasting marriages based on love with those based on financial security or social expectations.\n\n3. Social Class and Wealth: Austen examines the rigid class structure of Regency-era England and how it impacts relationships and social interactions.\n\n4. Family and Reputation: The importance of family connections and maintaining a good reputation in society is a recurring theme throughout the book.\n\n5. Gender Roles 